In [5]:
import numpy as np
from ondelette import *
import tensorflow as tf
import tensorflow_io as tfio

dtype = tf.float32
rng = np.random.default_rng()

In [6]:
song = 'MusicDelta_Rock'
audio = tfio.audio.AudioIOTensor(f'data/MedleyDB_V1/{song}_MIX.mp3', dtype=tf.float32)
rate = tf.cast(audio.rate, dtype=tf.int64)
raw_samples = audio.to_tensor()
samples = tf.transpose(raw_samples)
print(f'samples: {samples.shape[1]}')


samples: 577320


In [7]:
for name, make in wavelets.all.items():
    print(f'{name}...')

    wavelet = make(dtype=dtype)
    filters = wavelets.dwt_filters(wavelet)
    inverse_filters = wavelets.idwt_filters(wavelet)

    dec = wavelets.wavedec_same(filters, 11, samples)
    print(f'   DEC: {[l.shape[1] for l in dec]}')
    frames = framing.levels_to_frames(dec)
    print(f'  FRAMES: {[tuple(f.shape) for f in frames]}')
    slow = framing.stretch_levels(2, dec)
    slow = framing.stretch_frames(2, frames)
    print(f'  SLOW: {[l.shape[0] for l in slow]}')
    deframed = framing.frames_to_levels(slow)
    rec = wavelets.waverec(inverse_filters, deframed)
    print(f'   REC: {tuple(rec.shape)}')

    tx = tf.transpose(rec)
    mp3 = tfio.audio.encode_mp3(tx, rate)
    tf.io.write_file(f'temp/{name}.mp3', mp3)


bior1.1...
0 + 1128 + 0 -> 1128
0 + 2256 + 0 -> 2256
0 + 4511 + 1 -> 4512
0 + 9021 + 3 -> 9024
0 + 18042 + 6 -> 18048
0 + 36083 + 13 -> 36096
0 + 72165 + 27 -> 72192
0 + 144330 + 54 -> 144384
0 + 288660 + 108 -> 288768
   DEC: [564, 564, 1128, 2256, 4512, 9024, 18048, 36096, 72192, 144384, 288768]
  FRAMES: [(564, 2, 1), (564, 2, 1), (564, 2, 2), (564, 2, 4), (564, 2, 8), (564, 2, 16), (564, 2, 32), (564, 2, 64), (564, 2, 128), (564, 2, 256), (564, 2, 512)]
  SLOW: [1128, 1128, 1128, 1128, 1128, 1128, 1128, 1128, 1128, 1128, 1128]
   REC: (2, 1155072)
bior1.3...
4 + 1132 + 0 -> 1136
12 + 2260 + 0 -> 2272
28 + 4515 + 1 -> 4544
60 + 9025 + 3 -> 9088
124 + 18046 + 6 -> 18176
252 + 36087 + 13 -> 36352
508 + 72169 + 27 -> 72704
1020 + 144333 + 55 -> 145408
2044 + 288662 + 110 -> 290816
   DEC: [568, 568, 1136, 2272, 4544, 9088, 18176, 36352, 72704, 145408, 290816]
  FRAMES: [(568, 2, 1), (568, 2, 1), (568, 2, 2), (568, 2, 4), (568, 2, 8), (568, 2, 16), (568, 2, 32), (568, 2, 64), (568, 2, 1